In [65]:
from gurobipy import Model, GRB
import polars as pl

stocks = pl.read_csv("stocks.csv")

In [66]:
stocks.head()

Date,AAPL,GOOG,IBM,MARA,NVDA
str,f64,f64,f64,f64,f64
"""1/16/2023""",137.87,99.28,141.2,8.07,178.39
"""1/23/2023""",145.118851,100.709999,134.389999,8.02,203.649994
"""1/30/2023""",153.641205,105.220001,136.940002,7.07,211.0
"""2/6/2023""",150.170624,94.860001,135.600006,5.92,212.649994
"""2/13/2023""",151.933685,94.589996,135.020004,7.7,213.880005


In [67]:
# Get all the stock names to iterate over
stock_names = stocks.columns[1:]

# Calculate the weekly return percentage for each stock
stocks = stocks.with_columns(
    ((pl.col(s) - pl.col(s).shift(1)) / pl.col(s).shift(1) * 100.0).alias(s)
    for s in stock_names
).filter(pl.col("Date") != "1/16/2023")
stocks

Date,AAPL,GOOG,IBM,MARA,NVDA
str,f64,f64,f64,f64,f64
"""1/23/2023""",5.257744,1.44037,-4.822947,-0.619579,14.159983
"""1/30/2023""",5.872672,4.478207,1.897465,-11.845387,3.609136
"""2/6/2023""",-2.258887,-9.846037,-0.978528,-16.265912,0.781988
"""2/13/2023""",1.174039,-0.284635,-0.42773,30.067568,0.57842
"""2/20/2023""",-3.828253,-5.539696,-3.295806,-13.376623,8.874133
"""2/27/2023""",2.944577,5.226636,-0.712268,-4.347826,2.59383
"""3/6/2023""",-1.675162,-3.201441,-3.232029,-16.45768,-3.871913
"""3/13/2023""",4.377104,12.581031,-1.402945,53.283302,12.018292
"""3/20/2023""",3.387098,3.513565,1.293556,-4.406365,4.097185
